In [17]:
import pandas as pd

def make_nice_dataframe(csv_path, players_path, delete_missing=False):
    contest_df = pd.read_csv(csv_path)
    contest_df = contest_df[["Position", "Name", "Salary", "TeamAbbrev"]]
    players_df = pd.read_csv(players_path)

    # Add id column
    merged_df = pd.merge(contest_df, players_df, how='left', left_on='Name', right_on='full_name')

    # Print out rows with no match
    no_match_df = merged_df[merged_df['full_name'].isnull()]
    if not no_match_df.empty:
        print("Rows with no match:")
        print(no_match_df[["Name", "Salary"]])
    # Delete the no matches if set to do so
    if delete_missing:
        print("Deleting missing...")
        merged_df = merged_df.dropna(subset=['full_name'])

    merged_df["PG"] = False
    merged_df["SG"] = False
    merged_df["SF"] = False
    merged_df["PF"] = False
    merged_df["C"] = False
    merged_df["G"] = False
    merged_df["F"] = False
    merged_df["UTIL"] = True

    for idx, row in merged_df.iterrows():
        if "PG" in row["Position"]:
            merged_df.at[idx, "PG"] = True
        else:
            merged_df.at[idx, "PG"] = False
    
        if "SG" in row["Position"]:
            merged_df.at[idx, "SG"] = True
        else:
            merged_df.at[idx, "SG"] = False

        if "SF" in row["Position"]:
            merged_df.at[idx, "SF"] = True
        else:
            merged_df.at[idx, "SF"] = False

        if "PF" in row["Position"]:
            merged_df.at[idx, "PF"] = True
        else:
            merged_df.at[idx, "PF"] = False

        if "C" in row["Position"]:
            merged_df.at[idx, "C"] = True
        else:
            merged_df.at[idx, "C"] = False

        if "G" in row["Position"]:
            merged_df.at[idx, "G"] = True
        else:
            merged_df.at[idx, "G"] = False

        if "F" in row["Position"]:
            merged_df.at[idx, "F"] = True
        else:
            merged_df.at[idx, "F"] = False

    return merged_df

In [21]:
contest_csv = './games/157197163/DKSalaries.csv'
players_csv = './data/players.csv'
delete_missing = True

nice_df = make_nice_dataframe(contest_csv, players_csv, delete_missing=delete_missing)

Rows with no match:
                      Name  Salary
155  Juan Toscano-Anderson    3000
168       Brandon Williams    3000
186     Shaquille Harrison    3000
198             Vit Krejci    3000
204          Jontay Porter    3000
Deleting missing...


In [23]:
import json

bayes_simulations = './models/simpledfs.json'

with open(bayes_simulations, 'r') as f:
    bayes_data = json.load(f)

first_key = next(iter(bayes_data))
first_value = bayes_data[first_key]

print("First key:", first_key)
print("First 10 values of key:", first_value[:10])

First key: 2544
First 10 values of key: [39, 57, 43, 49, 61, 51, 65, 53, 62, 52]


In [40]:
import numpy as np

class TopPlayers:
    def __init__(self) -> None:
        self.pgs = []
        self.sgs = []
        self.sfs = []
        self.pfs = []
        self.cs = []
        self.gs = []
        self.fs = []
        self.utils = []

def add_top_position(means_dict, nice_df, position_name, top_num):

    position_players = nice_df[nice_df[position_name] == True]
    player_pairs = {}
    position_ids = list(position_players["player_id"])

    # If the mean key is in the position ids, add it to player_pairs
    for mean_key, mean_value in means_dict.items():
        if int(mean_key) in position_ids:
            player_pairs[mean_key] = mean_value

    # Get the top players in that position
    sorted_items = sorted(player_pairs.items(), key=lambda x: x[1], reverse=True)[:top_num]
    top_N_dict = dict(sorted_items)

    return top_N_dict.keys()

def get_top_of_positions(simulations_dict, nice_df, top_num):
    out = TopPlayers()

    means_dict = {}
    for id, sim_nums in simulations_dict.items():
        means_dict[id] = np.mean(sim_nums)
    
    out.pgs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="PG", top_num=top_num)
    out.sgs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="SG", top_num=top_num)
    out.sfs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="SF", top_num=top_num)
    out.pfs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="PF", top_num=top_num)
    out.cs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="C", top_num=top_num)
    out.gs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="G", top_num=top_num)
    out.fs = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="F", top_num=top_num)
    out.utils = add_top_position(means_dict=means_dict, nice_df=nice_df, position_name="UTIL", top_num=top_num)
    return out

players = get_top_of_positions(bayes_data, nice_df, 5)

In [44]:
for id in players.utils:
    print(id, np.mean(bayes_data[id]))

1629029 54.3831
203999 48.05445
201142 46.40685
1629027 46.28435
1626157 45.7787


In [45]:
class BatchLineupsIterator:
    def __init__(self, players: TopPlayers, batch_size: int):
        # TODO: Make this VALID
        self.starting_lineup = [players.pfs[0], players.sgs[0], players.sfs[0]]

    def __iter__(self):
        return self # leave this alone

    def __next__(self):
        # TODO: make a func that returns the next batch
        self.current += 1
        if self.current < self.high:
            return self.current
        raise StopIteration
    
    def getNextValidLineup(self):
        pass
    
batches = BatchLineupsIterator(3,9)

3
4
5
6
7
8
